In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('2017-06.csv')

In [6]:
df.head()

,timestamp,station_id,bikes,spaces,total_docks
20,2017-06-01 00:00:01,1,0,31,31
121,2017-06-01 00:02:01,1,0,31,31
222,2017-06-01 00:04:02,1,0,31,31
323,2017-06-01 00:06:02,1,0,31,31
424,2017-06-01 00:08:02,1,0,31,31


In [4]:
df.rename(columns={'tfl_id': 'station_id'}, inplace=True)

In [5]:
df = df.sort_values(by=['station_id','timestamp'])

In [7]:
df['deposit'] = df['bikes'].diff().fillna(0).astype(int)
df['bike_lag'] = df['bikes'].shift().fillna(0).astype(int)
df['bike_pull'] = df['bikes'].shift(-1).fillna(0).astype(int)

In [11]:
a = df['deposit'] != 0
x = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
df['cumm.deposits'] = x

#a = df['withdrawals'] != 0
#x = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
#df['cumm.withdrawals'] = x



## Resort and create DateTime variables

In [12]:
df = df.sort_values(by=['timestamp'])
df = df.set_index('timestamp')

In [13]:
df['hour'] = pd.DatetimeIndex(df.index).hour
df['day_of_week'] = pd.DatetimeIndex(df.index).dayofweek
df['month'] = pd.DatetimeIndex(df.index).month
df['time'] = pd.DatetimeIndex(df.index).time
df['time'] = df['time'].map(lambda x: x.replace(second=0))


In [14]:
#seeing what stations are over loaded and underloaded
df['overloaded'] = df['bikes'] == df['total_docks']
df['overloaded'] = df['overloaded'].astype(int)
df['underloaded'] = df['bikes'] == 0
df['underloaded'] = df['underloaded'].astype(int)

In [15]:
a = df['overloaded'] != 0
x = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
df['cumm.overloaded'] = x

a = df['underloaded'] != 0
x = a.cumsum()-a.cumsum().where(~a).ffill().fillna(0).astype(int)
df['cumm.underloaded'] = x

In [16]:
### pop bike_pull to the end (easier to have the target variable at the end)

In [17]:
pop_desposit = df.pop('bike_pull')
df['bike_pull'] = pop_desposit

In [22]:
df.columns

Index(['station_id', 'bikes', 'spaces', 'total_docks', 'deposit', 'bike_lag',
       'cumm.deposits', 'hour', 'day_of_week', 'month', 'time', 'overloaded',
       'underloaded', 'cumm.overloaded', 'cumm.underloaded', 'bike_pull'],
      dtype='object')

In [23]:
#get rid of saturday and sunday
df = df[df['day_of_week'] < 5]

In [24]:
end_remove = '05:30:00'
start_remove = '00:30:00'

df = df.loc[(df.index < start_remove) | (df.index > end_remove)]

## This is where we will create the files for 2018-05-02 with base and additional features

In [54]:
df.head()

,station_id,bikes,spaces,total_docks,deposit,bike_lag,cumm.deposits,hour,day_of_week,month,time,overloaded,underloaded,cumm.overloaded,cumm.underloaded,bike_pull
timestamp,,,,,,,,,,,,,,,,
2017-06-01 00:00:01,1,0,31,31,0,0,0,0,3,6,00:00:00,0,1,0,1,0
2017-06-01 00:00:01,82,22,0,22,18,4,1,0,3,6,00:00:00,1,0,1,0,22
2017-06-01 00:00:01,81,25,15,40,25,0,3,0,3,6,00:00:00,0,0,0,0,25
2017-06-01 00:00:01,80,25,15,40,25,0,1,0,3,6,00:00:00,0,0,0,0,26
2017-06-01 00:00:01,9,0,24,24,-28,28,1,0,3,6,00:00:00,0,1,0,1,0


In [50]:
y = df.loc['2018-06-27':'2018-06-28']
#x = y.iloc[:,15]

In [51]:
y = y.iloc[:,0:4]

In [55]:
y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 78480 entries, 2018-06-27 00:00:03 to 2018-06-27 23:58:02
Data columns (total 4 columns):
station_id     78480 non-null int64
bikes          78480 non-null int64
spaces         78480 non-null int64
total_docks    78480 non-null int64
dtypes: int64(4)
memory usage: 3.0+ MB


In [52]:
y.to_csv('2018-06-28.csv')

In [53]:
y.columns

Index(['station_id', 'bikes', 'spaces', 'total_docks'], dtype='object')

In [ ]:
df.loc['2018-01-01':'2018-06-28'].to_csv('2018-01-01 2018-06-28.csv')

In [30]:
#df.to_csv('2017-06_feat.csv')

In [31]:
#df = df.drop('bike_pull', axis=1)
#df = df.drop('minute', axis=1)